In [19]:
import pandas as pd
import folium
import branca.colormap as cm

In [20]:
dx = pd.read_csv('emersnow.csv')
dy = pd.read_csv('latlong.csv')

In [21]:
dx.head()

,date,location,time,type
0,2024-06-11,Orivesi,20:18:00,Rakennuspalo
1,2024-06-11,Kuopio,20:10:42,Rakennuspalo
2,2024-06-11,Pori,19:40:59,Tieliikenneonnettomuus
3,2024-06-11,Kruunupyy,19:35:16,Rakennuspalo
4,2024-06-11,Koski Tl,17:15:45,Rakennuspalo


In [22]:
dy.head()

,location,latitude,longitude
0,Akaa,61.167146,23.865889
1,Alajärvi,63.001273,23.817523
2,Alavieska,64.166646,24.307213
3,Alavus,62.586162,23.616601
4,Asikkala,61.172512,25.549142


In [23]:
df = pd.merge(dx, dy, on="location", how="left")
df.to_csv("merge1.csv", index=False)

In [24]:
df['incident_detail'] = df.apply(
    lambda row: f"{row['type']} ({row['date']} {row['time']})",
    axis=1
)
df.dropna(inplace=True)

In [25]:
df = df.drop_duplicates()
df

,date,location,time,type,latitude,longitude,incident_detail
0,2024-06-11,Orivesi,20:18:00,Rakennuspalo,61.677407,24.357389,Rakennuspalo (2024-06-11 20:18:00)
1,2024-06-11,Kuopio,20:10:42,Rakennuspalo,62.892983,27.688935,Rakennuspalo (2024-06-11 20:10:42)
2,2024-06-11,Pori,19:40:59,Tieliikenneonnettomuus,61.483726,21.795900,Tieliikenneonnettomuus (2024-06-11 19:40:59)
3,2024-06-11,Kruunupyy,19:35:16,Rakennuspalo,63.727818,23.033772,Rakennuspalo (2024-06-11 19:35:16)
4,2024-06-11,Koski Tl,17:15:45,Rakennuspalo,60.652982,23.137367,Rakennuspalo (2024-06-11 17:15:45)
...,...,...,...,...,...,...,...
454,2024-06-09,Valkeakoski,23:56:53,Tieliikenneonnettomuus,61.270053,24.029249,Tieliikenneonnettomuus (2024-06-09 23:56:53)
455,2024-06-09,Kuusamo,23:54:24,Rakennuspalo,65.964425,29.188616,Rakennuspalo (2024-06-09 23:54:24)
456,2024-06-09,Rauma,23:51:43,Öljy- tai ympäristöonnettomuus maalla,61.128738,21.511127,Öljy- tai ympäristöonnettomuus maalla (2024-06...
457,2024-06-09,Helsinki,23:45:26,Rakennuspalo,60.166641,24.943537,Rakennuspalo (2024-06-09 23:45:26)


In [38]:
max_date = df.date.max()
min_date = df.date.min()

title = f'{min_date} / {max_date}'

title_html = f'<h1 style="position:absolute;z-index:100000;left:40vw" >{title}</h1>'


In [39]:
all_locations = df.groupby('location').agg(
    latitude=('latitude', 'first'),
    longitude=('longitude', 'first'),
    incidents=('type', 'count'),
    incident_details=('incident_detail', lambda x: '<br>'.join(x))
).reset_index()

colormap = cm.LinearColormap(
    colors=['#00FFFF','#FFD700','#FF4500','#FF0000'],
    vmin=all_locations['incidents'].min(), 
    vmax=all_locations['incidents'].max()
)


incident_counts = df.groupby(['location', 'type']).size().unstack(fill_value=0).reset_index()

map = folium.Map(
    location=[64.9146659, 26.0672554], 
    tiles='cartodbdark_matter', 
    zoom_start=5
)
for i, row in all_locations.iterrows():
    location = [row['latitude'], row['longitude']]
    
    hover_text = (
        f"<h1 style='color:#696969'><u>{row['location']}</u></h1>"
        f"<i><small style='color:#696969'>Hälytykset: {row['incidents']}</small></i><br>" 
        ) 
    
    incident_info = incident_counts[incident_counts['location'] == row['location']].iloc[:, 1:]
    
    for incident_type, count in incident_info.items():
        if count.values[0] > 0:
            hover_text += f"<strong style='color:#FF4500'>{incident_type}:</strong> {count.values[0]}<br>"
    
    color = colormap(row['incidents'])
    
    popup_text = folium.Popup(f"<u>{row['incident_details']}</u>",
                              max_width=800
                              )
    
    folium.CircleMarker(
        location=location,
        radius=5,
        tooltip=hover_text + f"<small style='color:#696969'><b><i>lisätietoja...</b></i></small>",
        popup=popup_text,
        color=None, 
        fill=True,
        fill_color=color,  
        fill_opacity=0.6
    ).add_to(map)


map.get_root().html.add_child(folium.Element(title_html))

colormap.add_to(map)

# map.save('index.html')

map